# What is a Metaclass?

> « Metaclasses are deeper magic that 99% of users should never worry about. If you wonder whether you need them, you don't (the people who actually need them know with certainty that they need them, and don't need an explanation about why). »  
*Tim Peters, Python Guru*

Thank you, Tim! We're off to a good start!

This being said, it is not entirely false. It remains quite rare to create metaclasses in Python. However, they can solve complex problems elegantly.

Numerous libraries, such as Django, use them in their internal mechanics.

So what is a metaclass? It is a class whose instances are classes.

In other words, a class is a factory that creates variables called *class instances*. A metaclass is a factory that creates classes. Simply put, a factory that creates factories.

The advantage is to be able to benefit classes based on a metaclass of the same behaviour without having to copy/paste from one class to another. Inheritance was made for this. However, it does not solve everything...

## The limits of inheritance

Imagine you want to make a singleton class, which is a class that holds only one instance of a variable. If you create multiple instances, the class will always return the first one created, and every instance will hold the same **id**.

What is the purpose of such a class?

* The *None* constant is an example of a singleton. There exists only one copy at a time. This explains why the following can be written in tests:

```python
a == None or a is None
```

The **is** operator compares addresses. It works on this equality test because all instances of None have the same address. This would not be true for integers:

```python
a = 10**16
b = 10**16
print(a == b) # True
print(a is b) # Certainly False
```

* Peripheral drivers, such as the ones for printers, are often singleton objects considering there is no need for 50 copies of a driver stored in memory when only one is needed to communicates with the printer.
* Some software like *Word* are singletons. If you launch them several times, the instance currently active is re-displayed.

### How to implement a Singleton class in Python

Every language has its own techniques. In Java/C++, we can create a class with a private constructor to prevent it from being called by the developers and force them to pass a **get_instance** method that handles the class's unique constructor call.

In Python, everything is public. Therefore, the solution above does not apply. [Wikipédia provides](https://en.wikipedia.org/wiki/Singleton_pattern) a brief overview on how to implement the pattern. The page in [French](https://fr.wikipedia.org/wiki/Singleton_(patron_de_conception)) presents implementations in various languages, including Python.

To implement a singleton class, we must prevent the creation of multiple instances.

In Python, variables are created in the constructor. 

The constructor is not the `__init__` function.
The constructor is the function that creates the variable in memory.
`__init__` is called to initialize the instance's attributes once the variable is created. It is the [`__new__`](https://docs.python.org/3/reference/datamodel.html#object.__new__) function that allocates the object in memory.

We usually never define it because Python manages memory itself. And it is much simpler this way.

However, we could redefine it so that it:

* creates and stores a variable on the very first call
* returns the  already-created variable on subsequent calls

The unique singleton instance can be stored as a class attribute.
Which yields:

In [37]:
class Singleton(object):
    
    unique_instance = None  # Class attribute storing the single instance
    
    def __new__(cls, *args, **kwargs):
        """Notice the first parameter is not self but the class itself because
        the variable is not created yet. We will create it. 
        """
        
        # If the instance is not created yet...
        if cls.unique_instance is None:
            # We create it by calling the parent's __new__ method that knows how to do it...
            cls.unique_instance = super().__new__(cls, *args, **kwargs)
            
        return cls.unique_instance

All that is left to test is the following:

In [38]:
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton
print("Well played!")

Well played!


Mission well accomplished by using the `__new__` method rarely used in Python. This is starting to be high-level.

Now, let's imagine we need a second Singleton class. We won't duplicate the first but inherit from it considering the mechanics are already in place!

To do so, I duplicate the class above in notebook mode, which is the equivalent to running a script over to reinitialize `Singleton.instance_unique` to None.

In [39]:
class Singleton(object):
    
    unique_instance = None  # Class attribute storing the single instance
    
    def __new__(cls, *args, **kwargs):
        """Notice the first parameter is not self but the class itself because
        the variable is not created yet. We will create it. 
        """
        
        # If the instance is not created yet...
        if cls.unique_instance is None:
            # We create it by calling the parent's __new__ method that knows how to do it...
            cls.unique_instance = super().__new__(cls, *args, **kwargs)
            
        return cls.unique_instance
    
class SingletonChild(Singleton):
    pass


sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "I am not of the type expected"

print("Hehe... too easy... What is the need for a metaclass?")

Hehe... too easy... What is the need for a metaclass?


Too easy, or almost...

Let's imagine you need two singleton instances: one for the *Singleton* class and another for *SingletonChild*. You would thus write the following...

In [41]:
class Singleton(object):
    
    unique_instance = None  # Class attribute storing the single instance
    
    def __new__(cls, *args, **kwargs):
        """Notice the first parameter is not self but the class itself because
        the variable is not created yet. We will create it. 
        """
        
        # If the instance is not created yet...
        if cls.unique_instance is None:
            # We create it by calling the parent's __new__ method that knows how to do it...
            cls.unique_instance = super().__new__(cls, *args, **kwargs)
            
        return cls.unique_instance
    
class SingletonChild(Singleton):
    pass


# Creation of the singleton instance for the Singleton class
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton
print("Well played!")

# Creation of the singleton instance for the SingletonChild class
# HERE it's getting nasty...

sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "I am not of the type expected"

print("OUCH...")

Well played!


AssertionError: I am not of the type expected

In [42]:
assert isinstance(sc1, SingletonChild), "Even here..."

AssertionError: Even here...

In [43]:
print(type(sc1))
print(isinstance(sc1, Singleton))

<class '__main__.Singleton'>
True


In [44]:
print(sc1 == s1)

True


By inheriting SingletonChild from Singleton, the child class benefits from the same `__new__` function, but it has already allocated memory for the variable *s1* which is of the type Singleton.

As a result, SingletonChild does not have its own instance.

We would thus need to duplicate the `unique_instance` attribute in SingletonChild:

In [45]:
class SingletonChild(Singleton):
    unique_instance = None
    
    
sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)
assert isinstance(sc1, SingletonChild)
print("Ok, but we have duplicated a class attribute")

Ok, but we have duplicated a class attribute


The solution is viable and almost pretty, apart from the instance creation for each subclass of Singleton.

We can live well with this solution. However, we can do better by automatizing the code with metaclasses. It will avoid inheriting SingletonChild from Singleton for it to be a Singleton class...

## How classes are created in Python
Before embarking on the creation of metaclasses and classes based on this model, let's dive into a deeper understanding of the language and the creation of a class.

In Python, *everything is an object*.  
A function is an object, a class is an object...

In [46]:
def  hello():
    """Prints hello"""
    print("Hello!")
    
print(type(hello))

<class 'function'>


In [47]:
hi = hello
hi()

Hello!


What can we do with hello?

In [48]:
print(dir(hello))

['__annotations__', '__call__', '__class__', '__closure__', '__code__', '__defaults__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__get__', '__getattribute__', '__globals__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__kwdefaults__', '__le__', '__lt__', '__module__', '__name__', '__ne__', '__new__', '__qualname__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__']


In [49]:
print(hello.__doc__)   # The docstring is stored here
print(hello.__name__)  # The function's name

Prints hello
hello


AND you could also add attributes to the function, such as a call counter... But this goes beyond the article.

Types in Python such as *int*, *str*, and *list* are also objects. They are classes that are instances of metaclasses.

Therefore, what is the type of **int**? In other words, what is the metaclass that created the **int** class?
Is it **object**?   
No, **object** is the parent class of **int**, not the class that created **int**. It is the class **int** inherits from, just like *SingletonChild* inherits from *Singleton*. But *Singleton* did not create *SingletonChild*. *Singleton* created variables **s1** and **s2**.

In [27]:
print(type(int))

<class 'type'>


**Type** is the type of *int*. It is not just a function that displays a variable's type. It is a class that creates classes and thus a metaclass!

But who created **type**?

Can Python provide an answer to the why and how of the universe and of everything that is alive?

Is the answer *Guido Van Rossum* or *42*?
Let us wait no more for the answer. The answer to the Holy Grail of the existence of objects is there, all-ready! 
Let's ask the question!

In [50]:
print(type(type))  # Then?

<class 'type'>


It's *type*! *type* has created itself! It is the almighty class!

The *type* class can thus be used to create variables that are classes!

If we pass a single parameter to it, the variable's type is returned. Else, it will construct a class.

Here are the parameters accepted when constructing a class:

* The name of the class created
* A tuple containing the parents
* A dictionary containing the attributes and methods of the class

The following code:
    


In [51]:
class A(object):
    pass

class B(object):
    pass

MyClass = type("MyClass",
                (A, B),
                {"color" : 'blue', 
                 'hello' : lambda self: 'Hello man!'}
                )
mc = MyClass()
print(type(mc))
print(MyClass.color)
print(mc.hello())
print(MyClass.__mro__)

<class '__main__.MyClass'>
blue
Hello man!
(<class '__main__.MyClass'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


Is equivalent to the following:

In [52]:
class A(object):
    pass

class B(object):
    pass

class MyClass(A, B):
    color = "blue"
    
    def hello(self):
        return "Hello man!"

mc = MyClass()
print(type(mc))
print(MyClass.color)
print(mc.hello())
print(MyClass.__mro__)

<class '__main__.MyClass'>
blue
Hello man!
(<class '__main__.MyClass'>, <class '__main__.A'>, <class '__main__.B'>, <class 'object'>)


It's pretty bluffing at first. However, it is not very practical as defining methods through a dictionary is not that elegant.

In summary, when you write:

```python
class MaClasse(object):
    pass
```

Python executes:

```python
MaClasse = type("MaClasse", (object, ), {} )
```

Which remains valid, but it is more comfortable to use the keyword **class**. 

## How do we create a metaclass?

To do this, all that is needed is to create a class that inherits from **type**.

```python
class MyMetaClass(type):
    pass
```

A metaclass is a class that creates classes like *type*. Thus, we could write:

```python
MyClass = MyMetaClass()
my_object = MyClass()
```

Do not confuse:

* A variable's type: It is the name of the class that created the variable
* The parent of a class: It is the class from which your class inherits from, which can be consulted with `MyClass.__base__`

In [53]:
class MyMetaClass(type):
    pass

MyClass = MyMetaClass("MyClass", (object,), {})
my_object = MyClass()

print(type(MyClass))
print(type(my_object))
print(MyClass.__base__)

<class '__main__.MyMetaClass'>
<class '__main__.MyClass'>
<class 'object'>


But this syntax is not that elegant.
The [PEP3115](https://www.python.org/dev/peps/pep-3115/) presents the official syntax for defining a class based on a metaclass:

In [54]:
class MyMetaClass(type):
    pass

class MyClass(object, metaclass=MyMetaClass):
    pass

print(type(MyClass))
print(type(my_object))
print(MyClass.__base__)

<class '__main__.MyMetaClass'>
<class '__main__.MyClass'>
<class 'object'>


## How to implement the Singleton class with a metaclass ?

The idea is that if a class must have a `unique_instance` attribute, then all we need to do is initialize it in the metaclass. Thus, every class based on the metaclass will have this attribute without having the need to copy-paste the code.

### Details on the mechanics of creating objects in Python

When we write:

```python
my_variable = MyClass()
```

* Python allocates the object in memory with  the `MyClass.__new__` function.
* Then, it initializes the object the `MyClass.__init__` function.
  The `__init__` function allows assigning attributes to the created object.

But you know we can also call an object like a function if the object's class has the special `__class__` method defined.

For example:

In [55]:
class Vehicule:

    def __call__(self, *args, **kwargs):
        print("Off we go", *args)

v = Vehicule()
v("Paul", "Pierre", "Jacqueline")

Off we go Paul Pierre Jacqueline


To create a variable, you call the class like a normal function:

`variable = MyClass()` 

This calls the metaclass' `__call__` method by passing it the class as the first parameter:

* that creates the variable with `MyClass.__new__()`
* then initializes the variable with `MyClass.__init__()`
* then returns the variable 

Let's illustrate this with a concrete example...


In [56]:
print('Creation of the metaclass')
class MetaClass(type):
    
    def __new__(cls, *args, **kwargs):
        print("In MetaClass.__new__")
        return super().__new__(cls, *args, **kwargs)
    
    def __init__(cls, *args, **kwargs):
        print("In MetaClass.__init__", cls.__name__)
        super().__init__(*args, **kwargs)
    
    def __call__(cls, *args, **kwargs):
        print("In MetaClass.__call__", cls.__name__)
        return super().__call__(*args, **kwargs)
        
    
print("Creation of the class")
class MyClass(object, metaclass=MetaClass):

    def __new__(cls, *args, **kwargs):
        print("MyClass.__new__")
        return super().__new__(cls, *args, **kwargs)

    def __init__(self, *args, **kwargs):
        print("MyClass.__init__")
        super().__init__(*args, **kwargs)

    def __call__(self, *args, **kwargs):
        print("MyClass.__call__")
        print(*args, **kwargs)

print("Creation of the variable")
i = MyClass()
print("Calling the variable like a function")
i("What a beautiful mechanic...")

Creation of the metaclass
Creation of the class
In MetaClass.__new__
In MetaClass.__init__ MyClass
Creation of the variable
In MetaClass.__call__ MyClass
MyClass.__new__
MyClass.__init__
Calling the variable like a function
MyClass.__call__
What a beautiful mechanic...


**Note** the usage of *cls* when calling *super()* for the `__new__` method and without *cls* for the other methods such as `__init__`.
Indeed, the variable is not yet created when one calls `__new__` so you have to pass the attribute cls which does not represent the instance.

We can now write the metaclass **MetaSingleton** :

* When we'll create a class based on this metaclass, it's `__new__` et `__init__` methods will be called
* When we'll create a class instance based on the metaclass MetaSingleton, `MetaSingleton.__call__(Class)` will be called.
This is where you must test the unique instance's existence
* In the metaclass' `__init__` method, initialize the static attribute of the class. It will be easier this way.

In [57]:
class MetaSingleton(type):
    
    def __init__(cls, *args, **kwargs):
        """cls is the class created by the metaclass"""
        # add single instance class attribute
        cls.instance_unique = None

    def __call__(cls, *args, **kwargs):
        
        # We create a unique instance if does not already exist
        if cls.instance_unique is None:
            cls.instance_unique = super().__call__(*args, **kwargs)
            
        return cls.instance_unique
    
    
class Singleton(object, metaclass=MetaSingleton):
    pass


class SingletonChild(Singleton):
    pass


# Creating the singleton instances for the Singleton class
s1 = Singleton()
s2 = Singleton()

assert s1 == s2
assert s1 is s2
assert id(s1) == id(s2)
assert type(s1) == Singleton

# Everything will be fine now!
sc1 = SingletonChild()
sc2 = SingletonChild()

assert sc1 == sc2
assert sc2 is sc1
assert id(sc1) == id(sc2)

assert type(sc1) == SingletonChild, "I am not of the type expected"

print("Extraordinary")


Extraordinary


Here you are ready to build solid foundations...  
But very sincerely, we rarely create metaclasses, unless we develop sophisticated frameworks like the Django ORM.

If not professionally applied, this tutorial will have tried to give you an understanding of Python's internal mechanics.

And as *Tim Peters* says: **It's deep magic!**

To practice these concepts:

* Try implementing a metaclass that manages an object counter.
* Then, it decrements the counter once objects are destroyed...

Hum, is it possible?

## To look further into the subject

* A great [discussion thread about metaclasses on Stackoverflow](http://stackoverflow.com/questions/100003/what-is-a-metaclass-in-python)
* Another [interesting tutorial on metaclasses](https://blog.ionelmc.ro/2015/02/09/understanding-python-metaclasses/)
